# SCDAA Report

In [1]:
import numpy as np
from numpy.linalg import inv
from scipy.integrate import odeint, solve_ivp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim


## Exercise 1.1

In [2]:
class LQRController:
    def __init__(self, H, M, D, C, R, T):
        """
        Initialize the LQRController class with problem parameters.

        """
        self.H = H
        self.M = M
        self.D = D
        self.C = C
        self.R = R
        self.T = T

    def solve_riccati_ode(self, time_grid):
        
        def riccati_ode(t, S_flat):
            S = torch.tensor(S_flat).reshape((2, 2))  # Convert S_flat to a PyTorch tensor
            dS = -2 * torch.matmul(self.H.t(), S) + torch.matmul(S, torch.matmul(self.M, torch.matmul(torch.inverse(self.D), self.M.t()))) - self.C
            return dS.flatten().detach().numpy()

        S_final_flat = self.R.flatten().detach().numpy()
        sol = solve_ivp(riccati_ode, [self.T.item(), time_grid[0].item()], S_final_flat, t_eval=time_grid.numpy()[::-1], method='RK45')
        S_values = [torch.tensor(sol.y[:, i].reshape((2, 2))) for i in range(len(sol.t))]
        return S_values

    def optimal_control_value(self, time_grid, x_values):
        
        S_values = self.solve_riccati_ode(time_grid)
        control_values = []
        for S in S_values:
            S_tensor = torch.tensor(S)  # Convert S to a PyTorch tensor
            control = -torch.matmul(torch.inverse(self.D), torch.matmul(self.M.t(), torch.matmul(S_tensor, state_values.view(-1, 2).t()))).detach().unsqueeze(1)
            control_values.append(control)
        control_values = torch.cat(control_values, dim=1)
        return control_values

    def optimal_control_function(self, time_grid, x_values):
       
        S_values = self.solve_riccati_ode(time_grid)
        control_functions = []
        for S in S_values:
            S_tensor = torch.tensor(S)  # Convert S to a PyTorch tensor
            control = -torch.matmul(torch.inverse(self.D), torch.matmul(self.M.t(), torch.matmul(S_tensor, state_values.view(-1, 2).t()))).detach().t()
            control_functions.append(control)
        control_functions = torch.stack(control_functions)
        return control_functions

In [6]:
# Example Usage:
# Define problem matrices
H = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)
M = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)
D = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)
C = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)
R = torch.tensor([[1, 0], [0, 1]], dtype=torch.float64)
T = torch.tensor(1.0, dtype=torch.float64)

# Initialize LQRController
controller = LQRController(H, M, D, C, R, T)

# Define time grid
time_grid = torch.linspace(0, T, 100)

# Define state values
x_values = torch.tensor([[[0.5, 0.5]], [[1.0, 1.0]]], dtype=torch.float64)  # Example state values

# Calculate control problem value
control_value = controller.optimal_control_value(time_grid, x_values)
print("Control Problem Value:")
print(control_value)

Control Problem Value:
tensor([[[-0.5000, -1.0000],
         [-0.5102, -1.0203],
         [-0.5204, -1.0408],
         [-0.5308, -1.0615],
         [-0.5412, -1.0825],
         [-0.5518, -1.1036],
         [-0.5625, -1.1250],
         [-0.5733, -1.1465],
         [-0.5842, -1.1683],
         [-0.5952, -1.1903],
         [-0.6063, -1.2126],
         [-0.6175, -1.2350],
         [-0.6289, -1.2577],
         [-0.6403, -1.2807],
         [-0.6519, -1.3038],
         [-0.6636, -1.3272],
         [-0.6754, -1.3508],
         [-0.6874, -1.3747],
         [-0.6994, -1.3988],
         [-0.7116, -1.4232],
         [-0.7239, -1.4478],
         [-0.7363, -1.4727],
         [-0.7489, -1.4978],
         [-0.7616, -1.5231],
         [-0.7744, -1.5488],
         [-0.7873, -1.5746],
         [-0.8004, -1.6008],
         [-0.8136, -1.6272],
         [-0.8269, -1.6539],
         [-0.8404, -1.6808],
         [-0.8540, -1.7080],
         [-0.8678, -1.7355],
         [-0.8816, -1.7633],
         [-0.8957, -

/tmp/ipykernel_2931/1092302168.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  S_tensor = torch.tensor(S)  # Convert S to a PyTorch tensor


In [7]:
control_value.shape

torch.Size([2, 100, 2])

In [9]:

# Calculate control function
control_function = controller.optimal_control_function(time_grid, x_values)
print("\nControl Function:")
print(control_function)


Control Function:
tensor([[[-0.5000, -0.5000],
         [-1.0000, -1.0000]],

        [[-0.5102, -0.5102],
         [-1.0203, -1.0203]],

        [[-0.5204, -0.5204],
         [-1.0408, -1.0408]],

        [[-0.5308, -0.5308],
         [-1.0615, -1.0615]],

        [[-0.5412, -0.5412],
         [-1.0825, -1.0825]],

        [[-0.5518, -0.5518],
         [-1.1036, -1.1036]],

        [[-0.5625, -0.5625],
         [-1.1250, -1.1250]],

        [[-0.5733, -0.5733],
         [-1.1465, -1.1465]],

        [[-0.5842, -0.5842],
         [-1.1683, -1.1683]],

        [[-0.5952, -0.5952],
         [-1.1903, -1.1903]],

        [[-0.6063, -0.6063],
         [-1.2126, -1.2126]],

        [[-0.6175, -0.6175],
         [-1.2350, -1.2350]],

        [[-0.6289, -0.6289],
         [-1.2577, -1.2577]],

        [[-0.6403, -0.6403],
         [-1.2807, -1.2807]],

        [[-0.6519, -0.6519],
         [-1.3038, -1.3038]],

        [[-0.6636, -0.6636],
         [-1.3272, -1.3272]],

        [[-0.6754, -0

/tmp/ipykernel_2931/1092302168.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  S_tensor = torch.tensor(S)  # Convert S to a PyTorch tensor


In [ ]:
S_solution.shape

(4, 1000)

In [ ]:
# needed for exercise 2.1
class DGM_Layer(nn.Module):
    
    def __init__(self, dim_x, dim_S, activation='Tanh'):
        super(DGM_Layer, self).__init__()
        
        if activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'Tanh':
            self.activation = nn.Tanh()
        elif activation == 'Sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'LogSigmoid':
            self.activation = nn.LogSigmoid()
        else:
            raise ValueError("Unknown activation function {}".format(activation))
            

        self.gate_Z = self.layer(dim_x+dim_S, dim_S)
        self.gate_G = self.layer(dim_x+dim_S, dim_S)
        self.gate_R = self.layer(dim_x+dim_S, dim_S)
        self.gate_H = self.layer(dim_x+dim_S, dim_S)
            
    def layer(self, nIn, nOut):
        l = nn.Sequential(nn.Linear(nIn, nOut), self.activation)
        return l
    
    def forward(self, x, S):
        x_S = torch.cat([x,S],1)
        Z = self.gate_Z(x_S)
        G = self.gate_G(x_S)
        R = self.gate_R(x_S)
        
        input_gate_H = torch.cat([x, S*R],1)
        H = self.gate_H(input_gate_H)
        
        output = ((1-G))*H + Z*S
        return output


class Net_DGM(nn.Module):

    def __init__(self, dim_x, dim_S, activation='Tanh'):
        super(Net_DGM, self).__init__()

        self.dim = dim_x
        if activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'Tanh':
            self.activation = nn.Tanh()
        elif activation == 'Sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'LogSigmoid':
            self.activation = nn.LogSigmoid()
        else:
            raise ValueError("Unknown activation function {}".format(activation))

        self.input_layer = nn.Sequential(nn.Linear(dim_x+1, dim_S), self.activation)

        self.DGM1 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)
        self.DGM2 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)
        self.DGM3 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)

        self.output_layer = nn.Linear(dim_S, 1)

    def forward(self,t,x):
        tx = torch.cat([t,x], 1)
        S1 = self.input_layer(tx)
        S2 = self.DGM1(tx,S1)
        S3 = self.DGM2(tx,S2)
        S4 = self.DGM3(tx,S3)
        output = self.output_layer(S4)
        return output


In [ ]:
# needed for exercise 2.2
class FFN(nn.Module):

    def __init__(self, sizes, activation=nn.ReLU, output_activation=nn.Identity, batch_norm=False):
        super().__init__()
        
        layers = [nn.BatchNorm1d(sizes[0]),] if batch_norm else []
        for j in range(len(sizes)-1):
            layers.append(nn.Linear(sizes[j], sizes[j+1]))
            if batch_norm:
                layers.append(nn.BatchNorm1d(sizes[j+1], affine=True))
            if j<(len(sizes)-2):
                layers.append(activation())
            else:
                layers.append(output_activation())

        self.net = nn.Sequential(*layers)

    def freeze(self):
        for p in self.parameters():
            p.requires_grad=False

    def unfreeze(self):
        for p in self.parameters():
            p.requires_grad=True

    def forward(self, x):
        return self.net(x)


sizes = [input_size, 100, 100, output_size]  # Specify sizes of input, hidden, and output layers
ffn_model = FFN(sizes)